# Fundamentos de BD (Márcio Victorino)

## Partes do Projeto:
1. Introdução.
1. Modelo de dados Relacional.
1. O script SQL que gerou o banco de dados.
1. Apresentar o processo de ETL (Extract, Transform, Load) para importação dos
dados para o banco de dados.
1. Utilização de pelo menos uma View.
1. Utilização de pelo menos uma Procedure (com comandos condicionais).
1. Utilização de pelo menos um trigger (com comandos condicionais).
1. No mínimo 5 Consultas SQL (a complexidade da consulta será avaliada).

**Data Limite para a Entrega da Parte Escrita: 02/06/2023**

## Observações

- Dados abertos (ex: FNDE, bolsa família)
- Linkar com o Artigo da matéria do Ladeira MD
- Joga em um CSV e alimenta o BD SQL
- No mínimo milhares de dezenas de linhas
- Não executar a consulta na apresentação
- Printar os resultados e mostrar na apresentação

## Alunos: 

Marcelo Anselmo de Souza Filho
- Matrícula: **231109719**
- Email: **marcelofilho@mpf.mp.br**

Arivaldo Gonçalves de Freitas Junior
- Matrícula: **231109620**
- Email: **arivaldofreitas@correios.com.br**

Luciana Maria de Araujo Freitas
- Matrícula: **231109700**
- Email: **luciana@mpdft.mp.br**

____

# 1. Introdução

Este estudo aborda a diferença salarial entre homens e mulheres na área de TI durante a pandemia. Ele também explora possíveis cenários para analisara disparidade salarial e de desligamento entre gêneros na área de tecnologia, antes e após a pandemia.	Utilizou-se dados no nível do indivíduo, de 2019 e 2020, obtidos da Relação Anual de Informações Sociais (Rais), que proporciona dados oficiais sobre o mercado de trabalho no Brasil. No geral, constatou-se a remuneração média das mulheres é menor que a dos homens em todas as regiões do Brasil, tanto em 2019 quanto em 2020. Além disso, a quantidade de desligamento de homens é maior que a de mulheres em todas as regiões do Brasil, tanto em 2019 quanto em 2020.

# 2. Modelo de dados Relacional

## Carregar dados

In [ ]:
import pandas as pd

# # Mostrar mais colunas
pd.set_option("display.max_columns", 100)
pd.set_option('display.max_colwidth', 100)

df = pd.read_parquet(
    "../output/gold/rais_TODOS_ANOS_comJoin_RAIS_VINC_PUB.parquet.gzip")

qnt_full = len(df)
print(f"""
Quantidade docs: {qnt_full}
""")

df.head(2)


## Modelo Entidade Relacionamento

https://mermaid.js.org/syntax/entityRelationshipDiagram.html

Integrando jupyter com mermaid: https://mermaid.js.org/config/Tutorials.html#jupyter-integration-with-mermaid-js

In [75]:
import base64
from IPython.display import Image, display


def mm(graph):
  graph = graph.replace("\n", " ")
  graphbytes = graph.encode("ascii")
  base64_bytes = base64.b64encode(graphbytes)
  base64_string = base64_bytes.decode("ascii")
  display(Image(url="https://mermaid.ink/img/" + base64_string))



In [77]:
mm("""
erDiagram    
    RAIS ||--|| OCUPACAO : contem
    RAIS ||--|| OCUPACAO : contem
    RAIS ||--|| SEXO : contem
    RAIS ||--|| UF : contem
    UF ||--|| REGIAO : faz_parte
    RAIS {
        int id PK
        int id_ocupacao FK
        int id_sexo FK
        int id_uf FK
        int ano
        float remuneracao_media
        int desligamento
        int idade
    }
    OCUPACAO {
        int id PK
        string nome
    }
    SEXO {
        int id PK
        string nome
    }
    UF {
        int id PK
        int id_regiao FK
        string sigla_uf
    }
    REGIAO {
        int id PK
        string nome
    }
""")

## Normalização

# 3. O script SQL que gerou o banco de dados.

Conteúdo de: [querys_sql/GERAR_BD.sql](querys_sql/GERAR_BD.sql)

## Inserindo a RAIS (Tabela principal)

In [2]:
from __future__ import print_function
import pandas as pd
import mysql.connector
from datetime import date, datetime, timedelta


class CargaFullTabelaRAIS:
  def __init__(self, batch_size=1000, size_max=10000):
    self.batch_size = batch_size
    self.size_max = size_max
    self.cnx = mysql.connector.connect(
        user='root', password='root', database='projfbd')
    self.cursor = self.cnx.cursor()
    self.path_file_parquet = "../output/gold/rais_TODOS_ANOS_comJoin_RAIS_VINC_PUB.parquet.gzip"

  def carregar_CSV(self):
      # Mostrar mais colunas
    pd.set_option("display.max_columns", 100)
    pd.set_option('display.max_colwidth', 100)

    df = pd.read_parquet(self.path_file_parquet)

    qnt_total = len(df)

    print(f"""
      Quantidade de docs carregados: {qnt_total}
      """)
    return df

  def get_rais_from_pandas(self):
      df = self.carregar_CSV()
      counts = df[[
          "ocupacao_id", "sexo_id", "sigla_uf_id", "ano", "remuneracao_media", "desligamento", "idade"]]
      lista = counts.values.tolist()
      lista = [tuple(x) for x in lista]

      return lista

  def gerar_batch_insert_rais(self):
    print(f"""
      Inserindo os dados...
      """)

    rais_from_pandas = self.get_rais_from_pandas()

    for i in range(0, len(rais_from_pandas[:self.size_max]), self.batch_size):
      add_query = ("INSERT INTO RAIS "
                   "(id_ocupacao, id_sexo, id_uf, ano, remuneracao_media, desligamento, idade) "
                   "VALUES ")

      vals = ", ".join((f"({str(id_ocupacao)}, {str(id_sexo)}, {str(id_uf)}, {str(ano)}, {str(remuneracao_media)}, {str(desligamento)}, {str(idade)})")
                       for id_ocupacao, id_sexo, id_uf, ano, remuneracao_media, desligamento, idade in rais_from_pandas[i:i+self.batch_size])
      print(f"\n\n -- ====  INSERT a partir do dado {i}")
      # print(add_query + vals)
      try:
        self.cursor.execute(add_query + vals)
      except Exception as e:
        print(e)
        print(i)

    # ============ COMMIT ============
    self.cnx.commit()

  def init(self):
    self.gerar_batch_insert_rais()

  def __del__(self):
    self.cursor.close()
    self.cnx.close()


# ================= INICIANDO

CargaFullTabelaRAIS(batch_size=500000, size_max=2000000).init()



      Inserindo os dados...
      

      Quantidade de docs carregados: 1543009
      


 -- ====  INSERT a partir do dado 0


 -- ====  INSERT a partir do dado 500000


 -- ====  INSERT a partir do dado 1000000


 -- ====  INSERT a partir do dado 1500000


# 4. Apresentar o processo de ETL (Extract, Transform, Load) para importação dos dados para o banco de dados.

- Conteúdo de: [03_juntando_todos_anos.ipynb](../03_juntando_todos_anos.ipynb)
- Conteúdo de: [04_fazendo_join_ocup_sexo_raca.ipynb](../04_fazendo_join_ocup_sexo_raca.ipynb)

# 5. Utilização de pelo menos uma View.

Conteúdo de: [querys_sql/VIEW.sql](querys_sql/VIEW.sql)

# 6. Utilização de pelo menos uma Procedure (com comandos condicionais).

Conteúdo de: [querys_sql/PROCEDURE.sql](querys_sql/PROCEDURE.sql)

# 7. Utilização de pelo menos um trigger (com comandos condicionais).

Conteúdo de: [querys_sql/TRIGGER.sql](querys_sql/TRIGGER.sql)


# 8.  No mínimo 5 Consultas SQL (a complexidade da consulta será avaliada).

Conteúdo de: [querys_sql/CONSULTAS_COMPLEXAS.sql](querys_sql/CONSULTAS_COMPLEXAS.sql)
